In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, ReLU
import keras

In [2]:


#Set the path to your train dataset directory
dataset_path = './dataset/train'

# Define the image size for resizing
image_size = (32, 32)

# Load and preprocess the dataset
X_train = []
y_train = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_train.append(image)
                y_train.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

classes = np.unique(y_train)

#Set the path to your test dataset directory
dataset_path = './dataset/test'

# Load and preprocess the dataset
X_test = []
y_test = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_test.append(image)
                y_test.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_test = np.array(X_test)
y_test = np.array(y_test)

# Perform train-test split: This has already been done in the dataset
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

#Set the path to your Validation dataset directory
dataset_path = './dataset/val'

# Load and preprocess the dataset
X_val = []
y_val = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(dataset_path):
    shape_path = os.path.join(dataset_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)

                # Append the preprocessed image and corresponding label
                X_val.append(image)
                y_val.append(shape_dir)

# Convert the image data and labels to NumPy arrays
X_val = np.array(X_val)
y_val = np.array(y_val)


# Encode the shape labels into numerical values
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Convert the numerical labels to one-hot encoded vectors
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes)
y_val_one_hot = to_categorical(y_val_encoded, num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes)

# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))


# Check Predicted Shape

In [3]:
#Set the path to your test dataset directory
image_path = './dataset/new/'
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

In [4]:
print(classes)

In [5]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

# Training Model Using A tanh Activation In The Hidden Layer

In [6]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='tanh'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))

# Predict

In [7]:
#Set the path to your test dataset directory
image_path = './dataset/new/'
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

# Training Model Using A Sigmoid Activation In The Output Layer

In [8]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=10, batch_size=32, validation_data=(X_val, y_val_one_hot))

In [9]:
predicted_labels = []
# Define a new data point to classify
for i in range(10):
    j = i+1
    temp = os.path.join(image_path, 'newshape' +str(j)+'.jpg') #Current Image
    print(temp)
    
    new_data = np.array([cv2.resize(cv2.imread(temp), image_size)])

    # Predict the label for the new data point
    predicted_probabilities = model.predict(new_data)
    predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")

In [10]:
# Evaluate the model on the test data
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

# Print the accuracy
print("Accuracy:", accuracy)

# Initial Model Using ReLU in Input and Hidden Layer But Varied Epoch and Batch Size

The optimizer learning rate is changed.

In [ ]:
# Create the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64))
model.add(ReLU())
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train_one_hot, epochs=32, batch_size=64, validation_data=(X_val, y_val_one_hot))

# Predict Against Test Data

In [ ]:
test_path = './dataset/test'
predicted_labels = []

# Iterate through the shape directories in the dataset path
for shape_dir in os.listdir(test_path):
    shape_path = os.path.join(test_path, shape_dir)
    if os.path.isdir(shape_path):
        # Iterate through the images in each shape directory
        for image_file in os.listdir(shape_path):
            image_path = os.path.join(shape_path, image_file)
            if image_path.endswith('.jpg'):
                # Load the image and resize it to the desired size
                image = cv2.imread(image_path)
                image = cv2.resize(image, image_size)
                new_data = np.array([image])

                # Predict the label for the new data point
                predicted_probabilities = model.predict(new_data)
                predicted_labels.append(label_encoder.inverse_transform(np.argmax(predicted_probabilities, axis=1)))

# Iterate over the predicted labels
for predicted_label in predicted_labels:
    # Check if the predicted label exists in the list of available classes
    if predicted_label in classes:
        # Print the predicted label if it exists in the classes list
        print("Predicted label:", predicted_label)
    else:
        # Print "unmatched" if the predicted label does not match any class
        print("Unmatched")